In [5]:
import pymongo
import pandas as pd
import os
import sys
import json
import pickle
from sklearn.preprocessing import MinMaxScaler, binarize
from sklearn import metrics

In [6]:
MONGO_USER = "root"
MONGO_PASS = "rootPass"
MONGO_HOST = "lattice-101.cs.colostate.edu"
MONGO_PORT = 27018
MONGO_URL = f"mongodb://{MONGO_USER}:{MONGO_PASS}@{MONGO_HOST}:{MONGO_PORT}"
DB_NAME = "sustaindb"

FEATURE_FIELDS = [
    "PRESSURE_REDUCED_TO_MSL_PASCAL",
    "VISIBILITY_AT_SURFACE_METERS",
    "VISIBILITY_AT_CLOUD_TOP_METERS",
    "WIND_GUST_SPEED_AT_SURFACE_METERS_PER_SEC",
    "PRESSURE_AT_SURFACE_PASCAL",
]

# LABEL_FIELD = "CATEGORICAL_SNOW_SURFACE_BINARY"
LABEL_FIELD = "CATEGORICAL_RAIN_SURFACE_BINARY"

In [7]:
db_connection = pymongo.MongoClient(MONGO_URL)
db = db_connection[DB_NAME]
noaa = db['noaa_nam']

In [8]:
gis_joins = noaa.distinct('GISJOIN')
print(f'No. of GISJOINs: {len(gis_joins)}')
with open('./gis_joins.json', 'w') as out_file:
    out_file.write(str(gis_joins))

No. of GISJOINs: 3088
